In [109]:
from all_functions import *
from sklearn.metrics import mean_absolute_percentage_error as mape
from aeon.visualisation import plot_series
import pywt
from scipy import signal
from pyts.image import MarkovTransitionField
from pyts.image import GramianAngularField
from pyts.image import RecurrencePlot
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, RidgeCV, HuberRegressor, Lasso, LassoLarsCV
from aeon.transformations.collection.convolution_based import (
    MiniRocket,
    Rocket,
    MiniRocketMultivariateVariable,
)
import warnings

def rolling_window_image_concat(series, window, rep1, rep2, wavelet1, wavelet2, level1, level2):
  data = []
  for i in range(len(series)-window):
    example = np.array(series[i:i+window+1])
    target = example[-1]

    features = np.delete(example, -1)
    features_norm = znorm(features)
    
    target_norm = znorm_by(target, features)

    rep_features1 = transform_series(features_norm, rep1, wavelet1, level1)
    rep_features2 = transform_series(features_norm, rep2, wavelet2, level2)

    reps = np.concatenate((rep_features1.flatten(),rep_features2.flatten()))
    feat_target = np.concatenate((reps, [target_norm]))
    data.append(feat_target)
  df = pd.DataFrame(data)
  return df

In [110]:
import torch
import torch.nn as nn
import torch.optim as optim

class DilatedConvModel(nn.Module):
    def __init__(self, input_channels=1, num_filters=128, window_size=20):
        super(DilatedConvModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, num_filters, kernel_size=1, stride=1, padding=1)  # kernel_size 1
        self.conv2 = nn.Conv1d(num_filters, num_filters, kernel_size=2, stride=1, padding=1)
        self.conv3 = nn.Conv1d(num_filters, num_filters, kernel_size=2,  stride=1, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        
        # Ajuste o número de saídas na camada totalmente conectada
        self.fc = nn.Linear(num_filters * (window_size // 4), 1)

    def forward(self, x):
        print("Input shape:", x.shape)
        x = self.pool(torch.relu(self.conv1(x)))
        print("Shape after conv1 and pooling:", x.shape)
        x = self.pool(torch.relu(self.conv2(x)))
        print("Shape after conv2 and pooling:", x.shape)
        x = self.pool(torch.relu(self.conv3(x)))
        print("Shape after conv3 and pooling:", x.shape)
        x = x.view(x.size(0), -1)  # Achatar para a camada totalmente conectada
        x = self.fc(x)
        return x




# Função para transformar a série em tensores
def create_dataset(series, window_size=10):
    X, y = [], []
    for i in range(len(series) - window_size):
        X.append(series[i:i + window_size])  # Janela de entrada
        y.append(series[i + window_size])           # Valor a ser previsto
    return np.array(X), np.array(y)

In [111]:
# Função para ler os dados do arquivo TSF
metadata, series_data = read_tsf('../m4/m4_hourly_dataset.tsf')
for meta in metadata:
    if '@horizon' in meta:
        horizon = int(meta[9:])

series = series_data.loc[0]['series']
train_series, test_series = train_test_stats(pd.Series(series), horizon)

In [112]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

# Aumentando o tamanho da janela
window_size = 10  # Ajuste conforme necessário

# Criar dataset
X_train, y_train = create_dataset(train_series.tolist(), window_size)
X_test, y_test = create_dataset(test_series.tolist(), window_size)

# Converter para tensores do PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)  # Adicionando canal
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)  # Adicionando canal
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# Inicializar o modelo, critério e otimizador
model = DilatedConvModel(input_channels=1)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Treinamento do modelo
num_epochs = 500  # Ajuste conforme necessário
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)  # Passar dados pelo modelo
    loss = criterion(outputs, y_train_tensor)  # Calcular perda
    loss.backward()  # Retropropagação
    optimizer.step()  # Atualizar pesos
    if (epoch + 1) % 10 == 0:  # Exibir a cada 10 épocas
        print(f'Época {epoch+1}, Perda: {loss.item():.4f}')

# Avaliação do modelo
model.eval()  # Modo de avaliação
with torch.no_grad():  # Não calcular gradientes
    y_test_pred = model(X_test_tensor)

# Converter as previsões de volta para NumPy para comparação

def recursive_forecast(model, input_seq, horizon):
    model.eval()  # Modo de avaliação
    predictions = []
    
    with torch.no_grad():  # Não calcular gradientes
        for _ in range(horizon):
            # Fazer previsão
            input_tensor = torch.tensor(input_seq, dtype=torch.float32).unsqueeze(0).unsqueeze(1)  # Shape: [1, 1, window_size]
            pred = model(input_tensor)
            predictions.append(pred.item())  # Armazenar a previsão
            
            # Atualizar a sequência de entrada
            input_seq = np.append(input_seq[1:], pred.numpy())  # Deslocar a janela e adicionar a previsão
            
    return np.array(predictions)

last_input_seq = test_series[-window_size:].tolist()  # Últimos valores da série para iniciar a previsão
predictions = recursive_forecast(model, last_input_seq, horizon)

print(predictions)  # Exibir previsões

plt.figure(figsize=(12, 6))

# Plotando os valores reais
plt.plot(test_series.tolist(), label='Real', color='blue')

# Plotando as previsões
plt.plot(predictions, label='Valores Previsto', color='red')

plt.xlabel('Data')
plt.ylabel('Valores')
plt.title('Previsão da Série Temporal com Convoluções Dilatadas')
plt.legend()
plt.show()



Input shape: torch.Size([687, 1, 13])
Shape after conv1 and pooling: torch.Size([687, 128, 7])
Shape after conv2 and pooling: torch.Size([687, 128, 4])
Shape after conv3 and pooling: torch.Size([687, 128, 2])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (687x256 and 640x1)

In [100]:
test_series

700    619.0
701    565.0
702    532.0
703    495.0
704    481.0
705    467.0
706    473.0
707    488.0
708    501.0
709    534.0
710    576.0
711    639.0
712    712.0
713    772.0
714    830.0
715    880.0
716    893.0
717    896.0
718    891.0
719    854.0
720    803.0
721    769.0
722    751.0
723    701.0
724    635.0
725    572.0
726    532.0
727    493.0
728    477.0
729    468.0
730    464.0
731    477.0
732    492.0
733    519.0
734    568.0
735    624.0
736    696.0
737    761.0
738    812.0
739    836.0
740    838.0
741    829.0
742    807.0
743    785.0
744    756.0
745    719.0
746    703.0
747    659.0
dtype: float64